<a href="https://colab.research.google.com/github/serjduvk/Homework/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Part_1_Intro_to_SQL_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

Подключаем модуль для работы с базой SQLite

In [ ]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [ ]:
dir(sqlite3)

Создадим папку "my_databases", где будут хранится наши БД

In [ ]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
os.path.exists(path)

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [ ]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Заполнять таблицы можно тоже с помощью **execute().**

In [ ]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [ ]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [ ]:
conn.commit()
conn.close()

In [ ]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

In [ ]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

Почему на выходе получили только 1 комманду?

In [ ]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

In [ ]:
cursor.close()
conn.close()

In [ ]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

In [ ]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [ ]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [ ]:
rows

In [ ]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

In [ ]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

In [ ]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [3]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-14 17:10:26,948 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-14 17:10:26,950 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:10:26,952 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-14 17:10:26,955 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [4]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [6]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4543 sha256=f83392dc4b6caed068dcfbb094fd45c0997812826dbbf4c2d6c8b5059605d767
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [7]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [8]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [9]:
Purchase.ItemId.name

'ItemId'

In [10]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [11]:
Base.metadata.create_all(engine)

2020-12-14 17:11:28,512 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-14 17:11:28,517 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:11:28,523 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-14 17:11:28,524 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:11:28,525 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-14 17:11:28,526 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:11:28,527 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-14 17:11:28,528 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:11:28,531 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-14 17:11:28,533 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:11:28,535 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-14 17:11:28,536 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:11:28,541 INFO sqlalchemy.engine.base.Engine 
CREATE

In [12]:
engine.table_names()

2020-12-14 17:11:35,959 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-14 17:11:35,960 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [13]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-14 17:11:38,945 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:11:38,948 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 17:11:38,950 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-14 17:11:38,953 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [14]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-14 17:11:42,200 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 17:11:42,201 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [15]:
display_results(customers_query)

2020-12-14 17:11:46,018 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-14 17:11:46,020 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [16]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-14 17:11:51,209 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:11:51,212 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 17:11:51,213 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-14 17:11:51,217 INFO sqlalchemy.engine.base.Engine COMMIT


In [17]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-14 17:11:54,679 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:11:54,681 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 17:11:54,682 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-14 17:11:54,685 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-14 17:11:58,128 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:11:58,130 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-14 17:11:58,132 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-14 17:11:58,134 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [19]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-14 17:12:02,274 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-14 17:12:02,277 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com
3,Lisa,lisa@White.com


In [ ]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde45650938>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde45650990>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde45650570>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde45650888>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde45650a40>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde456509e8>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7fde45650468>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7fde45656780; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [20]:
from sqlalchemy import *

In [21]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email]).order_by(asc(Customer.FirstName), desc(Customer.LastName))
#customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-14 17:12:20,355 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC
2020-12-14 17:12:20,357 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [22]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-14 17:12:31,692 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC, customers."FirstName" DESC
2020-12-14 17:12:31,694 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [23]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-14 17:12:40,937 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-14 17:12:40,938 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [24]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))

In [25]:
display_results(customers_query)

2020-12-14 17:13:24,702 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-14 17:13:24,704 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [26]:
import pandas as pd

In [27]:
#выведем название нашей таблицы
Item.__tablename__

'items'

In [28]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-14 17:13:45,049 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-14 17:13:45,051 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:13:45,053 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-14 17:13:45,056 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:13:45,058 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-14 17:13:45,060 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [29]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
sql = """
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
"""
df = pd.read_sql(sql, conn)

df.head()

2020-12-14 17:14:54,396 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-14 17:14:54,398 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:14:54,400 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-14 17:14:54,403 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:14:54,406 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-14 17:14:54,407 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [30]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-14 17:15:17,708 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-14 17:15:17,710 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:15:17,711 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-14 17:15:17,714 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:15:17,715 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-14 17:15:17,716 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.


<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

In [33]:
хлеб=Item(
    Name='хлеб',
    Price=20)
session.add(хлеб)

jaket=Item(
    Name='jaket',
    Price=1020)
session.add(jaket)

printer=Item(
    Name='printer',
    Price=6000)
session.add(printer)

pc=Item(
    Name='PC',
    Price=51200)
session.add(pc)

laptop=Item(
    Name='laptop',
    Price=71020)
session.add(laptop)
session.commit()

2020-12-14 17:24:30,909 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:24:30,912 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 17:24:30,913 INFO sqlalchemy.engine.base.Engine ('хлеб', 20.0)
2020-12-14 17:24:30,915 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 17:24:30,917 INFO sqlalchemy.engine.base.Engine ('jaket', 1020.0)
2020-12-14 17:24:30,918 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 17:24:30,920 INFO sqlalchemy.engine.base.Engine ('printer', 6000.0)
2020-12-14 17:24:30,922 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 17:24:30,923 INFO sqlalchemy.engine.base.Engine ('PC', 51200.0)
2020-12-14 17:24:30,925 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-14 17:24:30,926 INFO sqlalchemy.engine.base.Engine ('laptop', 7102

In [34]:
query = select([Item])

display_results(query)

2020-12-14 17:24:36,814 INFO sqlalchemy.engine.base.OptionEngine SELECT items."ItemId", items."Name", items."Price" 
FROM items
2020-12-14 17:24:36,816 INFO sqlalchemy.engine.base.OptionEngine ()


/usr/local/lib/python3.6/dist-packages/sqlalchemy/sql/sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


,ItemId,Name,Price
0,1,хлеб,20.0
1,2,jaket,1020.0
2,3,printer,6000.0
3,4,PC,51200.0
4,5,laptop,71020.0


In [35]:
query = select([Customer])

display_results(query)

2020-12-14 17:24:45,020 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."CustomerId", customers."FirstName", customers."LastName", customers."Company", customers."Address", customers."Phone", customers."Email" 
FROM customers
2020-12-14 17:24:45,022 INFO sqlalchemy.engine.base.OptionEngine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [38]:
from datetime import datetime

In [39]:
perv=Purchase(
    ItemId=1,
    CustomerId=2,
    Date=datetime(2000,8,1)
)
session.add(perv)
session.commit()

2020-12-14 17:28:00,960 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:28:00,962 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 17:28:00,963 INFO sqlalchemy.engine.base.Engine (1, 2, '2000-08-01 00:00:00.000000')
2020-12-14 17:28:00,965 INFO sqlalchemy.engine.base.Engine COMMIT


In [40]:
vtor=Purchase(
    ItemId=2,
    CustomerId=3,
    Date=datetime(2020,3,15)
)
session.add(vtor)
dua=Purchase(
    ItemId=1,
    CustomerId=2,
    Date=datetime(2017,5,12)
)
session.add(dua)

trei=Purchase(
    ItemId=3,
    CustomerId=4,
    Date=datetime(2019,6,5)
)
session.add(trei)

patru=Purchase(
    ItemId=5,
    CustomerId=5,
    Date=datetime(2015,3,8)
)
session.add(patru)

session.commit()

2020-12-14 17:28:11,936 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-14 17:28:11,938 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 17:28:11,939 INFO sqlalchemy.engine.base.Engine (2, 3, '2020-03-15 00:00:00.000000')
2020-12-14 17:28:11,941 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 17:28:11,942 INFO sqlalchemy.engine.base.Engine (1, 2, '2017-05-12 00:00:00.000000')
2020-12-14 17:28:11,944 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 17:28:11,945 INFO sqlalchemy.engine.base.Engine (3, 4, '2019-06-05 00:00:00.000000')
2020-12-14 17:28:11,948 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-14 17:28:11,949 INFO sqlalchemy.engine.base.Engine (5, 5, '2015-03-08 00:00:00.000000')
2020-12-14 17:28

In [41]:
query = select(Customer.CustomerId,Customer.FirstName,Purchase.ItemId, Purchase.Date).where(Customer.FirstName == 'Lisa')
query = query.join (Purchase,Customer.CustomerId==Purchase.CustomerId)

display_results(query)

ArgumentError: ignored

In [42]:
df = pd.read_sql(
    '''SELECT * 
    FROM purchases p 
    join customers c on 
    p.customerid=c.customerid 
    where c.FirstName = 'Lisa'
    ''', conn)
df.tail()

2020-12-14 17:28:28,056 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * 
    FROM purchases p 
    join customers c on 
    p.customerid=c.customerid 
    where c.FirstName = 'Lisa'
    ")
2020-12-14 17:28:28,057 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:28:28,058 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * 
    FROM purchases p 
    join customers c on 
    p.customerid=c.customerid 
    where c.FirstName = 'Lisa'
    ")
2020-12-14 17:28:28,060 INFO sqlalchemy.engine.base.Engine ()
2020-12-14 17:28:28,061 INFO sqlalchemy.engine.base.Engine SELECT * 
    FROM purchases p 
    join customers c on 
    p.customerid=c.customerid 
    where c.FirstName = 'Lisa'
    
2020-12-14 17:28:28,062 INFO sqlalchemy.engine.base.Engine ()


,PurchaseId,ItemId,CustomerId,Date,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,1,2,2000-08-01 00:00:00.000000,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
1,3,1,2,2017-05-12 00:00:00.000000,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,4,3,4,2019-06-05 00:00:00.000000,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com
